In [46]:
from parseUtils import parseDocs

tagDict, tagList, textList, namesList = parseDocs()

with open('stop_words.txt') as sw:
    stop_words = [line.rstrip('\n') for line in sw]

with open('important_words.txt') as iw:
    imp_words = {line.rstrip('\n') for line in iw}

In [47]:
trainNum = 3684
testNumStart = 3685
testNumFinish = len(textList)

In [58]:
from nltk.stem.snowball import SnowballStemmer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

stemmer = SnowballStemmer('russian')
tkzr = CountVectorizer().build_tokenizer()

stem_tokenize = lambda tokens: [stemmer.stem(item) for item in tokens if not item.isdigit()]
tokenize = lambda text: stem_tokenize(tkzr(text))

prepoc = Pipeline([('count', CountVectorizer(tokenizer=tokenize,
                                             max_df=0.8,
                                             min_df=0.01,
                                             ngram_range=(1, 2),
                                             stop_words = stem_tokenize(stop_words))),
                   ('tfidf', TfidfTransformer())])

In [59]:
from sklearn.preprocessing import MultiLabelBinarizer

X_train = prepoc.fit_transform(textList)
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(tagList)

In [76]:
from parseUtils import parseDoc
p = '/Volumes/Media/Documents/Git/MachineLearning/out/21.txt'
tags, text = parseDoc(p)
textList.append(text)
X_train = prepoc.fit_transform(textList)

In [77]:
from sklearn.feature_selection import (chi2, SelectKBest)
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

clf = Pipeline([('chi2', SelectKBest(chi2, k = 'all')),
                ('svm', LinearSVC(random_state=0,
                                  class_weight='balanced',
                                  C=2.0))])

classer = OneVsRestClassifier(clf, n_jobs=1).fit(X_train[:trainNum, :],
                                                y[:trainNum, :])

In [11]:
import evaluation
out = ("/Volumes/Media/Documents/Git/MachineLearning/src"
            "/main/resources/classifierOuts/")
evaluation.eval(out_path = out, 
                classifier = classer, 
                mlb = mlb,
                x_train = X_train,
                tagList = tagList,
                namesList = namesList).evalTrainer(testNumStart, testNumFinish)


macro_precision = 0.6536373507057546
micro_precision = 0.8040816326530612
recall = 0.7748279252704031
F1 = 0.7891837756634953
macro_correct = 602
micro_correct = 788
false_neg_tags = 229
false_pos_tags = 192
total_test_docs = 921
n_of_pred_tags = 980
n_of_relevant_tags = 1017


In [84]:
l = ['a', 'b']
for i in range(0, len(l)):
    print(l)

['a', 'b']
['a', 'b']


In [117]:
from bs4 import BeautifulSoup as soup
def update_file(f, tags):
    with open(f, 'r') as fl:
        dom = soup(fl, 'lxml')
        t = dom.find('tags')
        t.clear()
        for tag in tags:
            ta = dom.new_tag('tag')
            t.append(ta)
            ta.string = tag
        with open(f, 'w') as fll:
            html = soup.prettify(dom)
            fll.write(html)

In [121]:
perdicted = classer.predict(X_train[X_train.shape[0] - 1])
tags_pred = list()
for j in predicted.nonzero()[1]:
    tags_pred.append(mlb.classes_.item(j))
if tags_pred:
    update_file(p, tags_pred)